
# Market Impact — Almgren–Chriss (AC) Optimal Execution
**Created:** 2025-08-31 20:27:40

This notebook implements the discrete-time **Almgren–Chriss (2001)** optimal execution model.
You can compute the **optimal trading trajectory** that minimizes expected cost + risk
(variance of cost), visualize schedules, and compare against **TWAP**/**VWAP-like** baselines.

### Features
- Discrete-time AC model with **temporary** (η) and **permanent** (γ) impact
- Objective: minimize \( \mathbb E[C] + \lambda \operatorname{Var}(C) \)
- Solve for optimal **shares remaining** trajectory \(x_t\) via a **convex QP closed-form linear system**
- Compare with **TWAP**
- **Efficient frontier** by sweeping risk aversion \(\lambda\)
- Optional execution **simulation** under price noise to inspect ex-post slippage

> Notes:
> - Units: shares for sizes, seconds or minutes for time, price in currency units.
> - Choose a coherent time step Δt (e.g., 60s) and calibrate impact/vol accordingly.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass

np.set_printoptions(precision=6, suppress=True)
pd.options.display.float_format = '{:,.6f}'.format



## 1) Model Setup
We consider a sell program of \(X_0\) shares over horizon \(T\), split into \(N\) equal intervals of \(\Delta t = T/N\).  
Let \(x_t\) be **shares remaining** right before trading in period \(t\) (so \(x_0=X_0, x_N=0\)).  
Trading rate \(v_t = (x_t - x_{t+1})/\Delta t\).

**Impact & Risk model (discrete):**

- Expected cost:
\[
\mathbb E[C] = \sum_{t=0}^{N-1} \big( \gamma\,x_t (x_t - x_{t+1}) + \frac{\eta}{\Delta t}(x_t - x_{t+1})^2 \big)
\]
- Variance of cost (due to price diffusion with volatility \(\sigma\)):
\[
\operatorname{Var}(C) = \sigma^2 \sum_{t=0}^{N-1} x_t^2\, \Delta t
\]
- Objective to minimize: \( J = \mathbb E[C] + \lambda\,\operatorname{Var}(C) \).

This yields a **strictly convex quadratic** objective in variables \(x_1,\ldots,x_{N-1}\) with fixed \(x_0=X_0, x_N=0\).
We solve the normal equations for the interior points via a tridiagonal linear system.


In [ ]:

from dataclasses import dataclass

@dataclass
class ACParams:
    X0: float = 1_000_000      # shares to sell
    T: float = 3600.0          # horizon in seconds (e.g., 1 hour)
    N: int = 60                # number of slices
    eta: float = 1e-6          # temporary impact coefficient (currency·time per share^2)
    gamma: float = 1e-7        # permanent impact coefficient (currency per share)
    sigma: float = 0.02        # price volatility per sqrt(time_unit of Δt)
    lam: float = 1e-6          # risk aversion λ

def build_Q_matrix(params: ACParams):
    N = params.N
    Δ = params.T / params.N
    η = params.eta
    γ = params.gamma
    σ = params.sigma
    λ = params.lam

    size = N + 1
    Q = np.zeros((size, size), dtype=float)

    for t in range(N):
        Q[t, t] += (γ + η/Δ + λ * σ**2 * Δ)
        Q[t, t+1] += ( -γ - 2*η/Δ ) / 2.0
        Q[t+1, t] = Q[t, t+1]
        Q[t+1, t+1] += (η/Δ)

    return Q

def solve_optimal_trajectory(params: ACParams):
    Q = build_Q_matrix(params)
    N = params.N
    X0 = params.X0

    interior = np.arange(1, N)
    boundary_idx = np.array([0, N])
    bvals = np.array([X0, 0.0])

    Q_ii = Q[np.ix_(interior, interior)]
    Q_ib = Q[np.ix_(interior, boundary_idx)]

    rhs = - Q_ib @ bvals
    x_interior = np.linalg.solve(Q_ii, rhs)

    x = np.zeros(N+1)
    x[0] = X0
    x[1:N] = x_interior
    x[N] = 0.0
    Δ = params.T / params.N
    v = (x[:-1] - x[1:]) / Δ
    q = x[:-1] - x[1:]
    return x, v, q

def evaluate_cost_variance(x, params: ACParams):
    X0, T, N = params.X0, params.T, params.N
    Δ = T / N
    η, γ, σ, λ = params.eta, params.gamma, params.sigma, params.lam

    EC = 0.0
    Var = 0.0
    for t in range(N):
        d = x[t] - x[t+1]
        EC += γ * x[t] * d + (η/Δ) * (d**2)
        Var += (x[t]**2) * σ**2 * Δ
    J = EC + λ * Var
    return EC, Var, J

def twap_trajectory(params: ACParams):
    N = params.N
    x = np.linspace(params.X0, 0.0, N+1)
    Δ = params.T / params.N
    v = (x[:-1] - x[1:]) / Δ
    q = x[:-1] - x[1:]
    return x, v, q



## 2) Example: Optimal Trajectory vs. TWAP
Edit the parameters and re-run to see how risk aversion (λ), volatility (σ), and impact (η, γ) change the schedule.


In [ ]:

p = ACParams(
    X0=1_000_000,
    T=3600.0,
    N=60,
    eta=5e-7,
    gamma=1e-7,
    sigma=0.004,
    lam=2e-6
)

x_opt, v_opt, q_opt = solve_optimal_trajectory(p)
EC_opt, Var_opt, J_opt = evaluate_cost_variance(x_opt, p)

x_twap, v_twap, q_twap = twap_trajectory(p)
EC_twap, Var_twap, J_twap = evaluate_cost_variance(x_twap, p)

print("=== Costs (currency units) ===")
print(f"Optimal:  E[C]={EC_opt:,.2f}  Var[C]={Var_opt:,.2f}  J={J_opt:,.2f}")
print(f"TWAP:     E[C]={EC_twap:,.2f}  Var[C]={Var_twap:,.2f}  J={J_twap:,.2f}")


In [ ]:

plt.figure(figsize=(9,5))
plt.plot(x_opt, linewidth=2, label="Optimal")
plt.plot(x_twap, linewidth=1, linestyle="--", label="TWAP")
plt.title("Shares Remaining (x_t)")
plt.xlabel("Slice t")
plt.ylabel("Shares")
plt.legend()
plt.show()


In [ ]:

plt.figure(figsize=(9,5))
plt.plot(q_opt, linewidth=2, label="Optimal")
plt.plot(q_twap, linewidth=1, linestyle="--", label="TWAP")
plt.title("Shares Traded per Slice (q_t)")
plt.xlabel("Slice t")
plt.ylabel("Shares")
plt.legend()
plt.show()



## 3) Efficient Frontier (λ sweep)
We vary risk aversion \(\lambda\) over a grid and compute \((\sqrt{\operatorname{Var}(C)}, \mathbb E[C])\) to visualize the frontier.


In [ ]:

def efficient_frontier(params: ACParams, lambdas):
    ECs, VARs = [], []
    for lam in lambdas:
        p2 = ACParams(**{**params.__dict__, "lam": float(lam)})
        x, _, _ = solve_optimal_trajectory(p2)
        EC, Var, _ = evaluate_cost_variance(x, p2)
        ECs.append(EC); VARs.append(Var)
    return np.array(ECs), np.array(VARs)

lams = np.logspace(-8, -3, 20)
ECs, VARs = efficient_frontier(p, lams)

plt.figure(figsize=(8,5))
plt.scatter(np.sqrt(VARs), ECs, s=22)
plt.title("Efficient Frontier: sqrt(Var(C)) vs E[C]")
plt.xlabel("Risk (sqrt Var)")
plt.ylabel("Expected Cost")
plt.show()



## 4) Optional Execution Simulation
Simulate price paths with diffusion and apply impact to inspect realized slippage for a chosen schedule.
This is illustrative and not a full microstructure simulator.


In [ ]:

def simulate_execution(x, params: ACParams, n_paths: int = 200, S0: float = 100.0, seed: int = 123):
    rng = np.random.default_rng(seed)
    N = params.N
    Δ = params.T / N
    η, γ, σ = params.eta, params.gamma, params.sigma
    q = x[:-1] - x[1:]

    costs = np.zeros(n_paths)
    for k in range(n_paths):
        S = np.zeros(N+1); S[0] = S0
        for t in range(N):
            S[t+1] = S[t] + σ * np.sqrt(Δ) * rng.standard_normal() + γ * q[t]
        P_exec = S[:-1] + η * (q / Δ)
        costs[k] = (q * P_exec).sum() - params.X0 * S0
    return costs

opt_costs = simulate_execution(x_opt, p, n_paths=200)
twap_costs = simulate_execution(x_twap, p, n_paths=200)

print(f"Simulated realized cost — Optimal: mean={opt_costs.mean():,.2f}, std={opt_costs.std(ddof=1):,.2f}")
print(f"Simulated realized cost — TWAP:    mean={twap_costs.mean():,.2f}, std={twap_costs.std(ddof=1):,.2f}")


In [ ]:

plt.figure(figsize=(9,5))
plt.hist(opt_costs, bins=40, alpha=0.7, label="Optimal")
plt.hist(twap_costs, bins=40, alpha=0.5, label="TWAP")
plt.title("Distribution of Realized Costs (Simulation)")
plt.xlabel("Cost")
plt.ylabel("Frequency")
plt.legend()
plt.show()



## 5) Calibration Tips
- **σ (volatility):** estimate from intraday returns at your chosen Δt; ensure units match T/N.
- **η (temporary impact):** fit from short-horizon regressions of price slippage vs. trade rate.
- **γ (permanent impact):** fit from price moves vs. net signed volume across windows.
- Use a **consistent Δt** (e.g., 1 minute) across σ, η, γ.
- Check sensitivity by sweeping parameters to ensure robust schedules.
